In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

In [2]:
import os, sys, time

In [3]:
import ROOT

Welcome to JupyROOT 6.10/09


In [4]:
from importlib import import_module
from PhysicsTools.NanoAODTools.postprocessing.framework.postprocessor import PostProcessor

In [5]:
from PhysicsTools.NanoAODTools.postprocessing.framework.datamodel import Collection, Object
from PhysicsTools.NanoAODTools.postprocessing.framework.eventloop import Module

In [53]:
class Jetsons(Module):
    def __init__(self):
        self.writeHistFile=True
        self.counter = 0
    def beginJob(self,histFile=None,histDirName=None):
        Module.beginJob(self,histFile,histDirName)
        
        self.h_lumiblock = ROOT.TH1I('h_lumiblock', ';LumiBlock;Events',   100, 0, 1000)
        self.addObject(self.h_lumiblock)
        self.h_run = ROOT.TH1D('h_run', ';Run;Events', 10, 273400, 277000) #Double, because I is 4 bytes, run is 64bit UInt
        self.addObject(self.h_run)
    #def endJob(self):
    #    pass #Cannot do pass in endJob if we write histograms... they don't get written! So do no override Module 
    #def beginFile(self, inputFile, outputFile):
    #def beginFile(self):
        #pass
    #def endFile(self, inputFile, outputFile):
    #def endFile(self):
        #pass
    def analyze(self, event):
        """process event, return True (go to next module) or False (fail, go to next event)"""
        
        modulator = 1000 # how often to print an event when we do self.counter % modulator
        self.counter += 1
        if self.counter > 10:
            return False
        ###########################################
        ###### Basic Attributes of the Event ######
        ###########################################
        run = getattr(event, "run")
        lumi = getattr(event, "luminosityBlock")
        evt = getattr(event, "event")
        
        self.h_lumiblock.Fill(lumi)
        self.h_run.Fill(float(run))
        #if self.counter % modulator == 0:
        print("\n\nRun: {0:>8d} LuminosityBlock: {1:>8d} Event: {2:>8d}".format(run,lumi,evt)) 
    
        ###########################################
        ###### Event Collections and Objects ######
        ###########################################
        #electrons = Collection(event, "Electron")
        #photons = Collection(event, "Photon")
        muons = Collection(event, "Muon")
        #taus = Collection(event, "Tau")
        jets = Collection(event, "Jet")
        fatjets = Collection(event, "FatJet")
        subjets = Collection(event, "SubJet")
        met = Object(event, "MET")
        PV = Object(event, "PV")
        SV = Collection(event, "SV")
        
        ###############################
        ###### Time To Do Stuff? ######
        ###############################
        eventSum = ROOT.TLorentzVector()
        print("PV  X: {0: >5.3f} Y: {1: >5.3f} Z: {2:5.3f} nDoF: {3: >5f} Chi^2: {4: >5.3f}".format(
            PV.x,PV.y, PV.z, PV.ndof, PV.chi2)) #getattr(PV, "chi2") also works
        print("==============================================================================")
        print("|| Secondary Vertices\tNumber: {0: >3d} \t\t\t\t\t    ||".format(len(SV)))
        if len(SV) > 0:
            print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t{5:>5s}\t{6:>5s}\t\t||".format(
                "Pt", "Eta", "Phi", "Chi2", "nDoF", "Mass", "dLen"))
            for vert in SV:
                print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t{5: >5.3f}\t{6: >5.3f}\t\t    ||".format(
                    getattr(vert,"pt"),getattr(vert,"eta"),getattr(vert,"phi"),
                    getattr(vert,"chi2"),getattr(vert,"ndof"),getattr(vert,"mass"),getattr(vert,"dlen")))
        print("==============================================================================")
        print("||Muons\tNumber: {0: >3d}\t\t\t\t\t\t    ||".format(len(muons)))
        print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t{5:>5s}\t{6:>5s}\t{7:>5s}\t||".format(
                "Pt", "Eta", "Phi", "IP3d", "dXY", "dZ", "JetID", "PFRelIso04"))
        for lep in muons :
            eventSum += lep.p4()
            print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t{5: >5.3f}\t{6: >5d}\t{7: >5.3f}\t    ||".format(
                    getattr(lep,"pt"),getattr(lep,"eta"),getattr(lep,"phi"),getattr(lep,"ip3d"),
                    getattr(lep,"dxy"),getattr(lep,"dz"),getattr(lep,"jetIdx"),getattr(lep,"pfRelIso04_all")))
        print("==============================================================================")
        print("||Electrons\tNumber: {0: >3d}\t\t\t\t\t\t    ||".format(len(muons)))
        print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t{5:>5s}\t{6:>5s}\t{7:>5s}\t||".format(
                "Pt", "Eta", "Phi", "IP3d", "dXY", "dZ", "JetID", "PFRelIso03"))
        for nm, lep in enumerate(muons) :
            eventSum += lep.p4()
            print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t{5: >5.3f}\t{6: >5d}\t{7: >5.3f}\t    ||".format(
                    getattr(lep,"pt"),getattr(lep,"eta"),getattr(lep,"phi"),getattr(lep,"ip3d"),
                    getattr(lep,"dxy"),getattr(lep,"dz"),getattr(lep,"jetIdx"),getattr(lep,"pfRelIso03_all")))
        print("==============================================================================")
        return True #Test by ending early
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Electron", "Pt", "Eta", "Phi"))
        for ne, lep in enumerate(electrons) :
            eventSum += lep.p4()
            print("{0:*^5d} {1:>10.4f} {2:> 10.3f} {3:> 10.3f}".format(ne+1, lep.pt, lep.eta, lep.phi))
        #for j in filter(self.jetSel,jets):
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Jet", "Pt", "Eta", "Phi"))
        for nj, j in enumerate(jets):
            eventSum += j.p4()
            print("{0: >5d} {1:>10.4f} {2:>-10.3f} {3:>-10.3f}".format(nj+1, j.pt, j.eta, j.phi))
        print("Event Mass: {:<10.4f}\n".format(eventSum.M()))
        
        ###########################################
        ###### Return True to pass the event ######
        ###########################################
        return True


In [54]:
#preselection="nMuon > 0 && nElectron > 0"
outputDir = "." #Just output where we run from for now
preselection=None
filePrefix = "root://cms-xrd-global.cern.ch/"
inputList = open("../data/Run2016/Run2016B_SM", "r")
files=[]
for line in inputList:
    files.append(filePrefix + str(line))
#for file in files:
#    print(file)
onefile = [files[0]] #Experiment with just one file for now...
#onefile = " root://cms-xrd-global.cern.ch//store/data/Run2016B/SingleMuon/NANOAOD/05Feb2018_ver2-v1/00000/2055267F-3110-E811-8956-AC1F6B1AEF94.root"
print(onefile)

[u'root://cms-xrd-global.cern.ch//store/data/Run2016B/SingleMuon/NANOAOD/05Feb2018_ver2-v1/00000/2055267F-3110-E811-8956-AC1F6B1AEF94.root\n']


In [55]:
p=PostProcessor(outputDir,onefile,cut=preselection,branchsel=None,modules=[Jetsons()],friend=False,postfix="_Jetsons", 
                jsonInput="../data/json/2016Golden.json",noOut=False,justcount=False,provenance=False,
                haddFileName=None,fwkJobReport=False,histFileName="histOut.root",
                histDirName="plots", outputbranchsel=None)
#"." is the outputdirectory, i.e. "Here" in this case
#jsonInput: json file in dictionary format {"RunNumberInt": [[lumilow,lumihigh],[lumi2low,lumi2high]], "RunNumber2Int":[[low,high]]}
#files: list ["fileone.root","filetwo.root"] of inputs, even if one file must be a list!
#branchsel: if non-None, selection of branches to not even activate/load into memory
#outputbranchsel: if non-None, selection of branches to still include in output (see noOut)
#noOut: If True, no output of skimmed data is written. If False, will write full data file fitting outputbranch selections and postfix concattenated to name
#postfix: string added to inputfile name to indicate this module processed it!
#histFileName: name of any output file for histograms created in your class, as above with runs and lumis
#histDirName: name INSIDE the "histFileName.root" file's directory structure!
#friend: Not tested
#provenance: not tested
#haddFileName: Not properly tested (tied together with fwkJobReport)
#fwkJobReport: only relevant with multiple files being added together? Not properly tested

In [ ]:
p.run()

Will write selected trees to .
Pre-select 11647 entries out of 11647 
Run:   273410 LuminosityBlock:       50 Event: 84373408
